# Data Structure and Interpreter of Relational Algebra

Motivation (Anecdote). One day, I read the chapter 4 of the database textbook on relational algebra and it has a section giving examples of queries using relational algebra. However, I'm very careless in calculating the algebra in calculating the queries that I formulated. Since it's algebra, instead of calculating these by hand, why not implement this as an interpreter? where I only type the query and the interpreter return the result. 

The interpreter is indeed, resemblance to SQL engine.  To implement this interpreter, it requires data structure to represent table and provide some primitive operators. This notebook provides the naive implementation of these.

By implementing the toy SQL interpreter, we can see that how's `equijoin` (i.e.: inner join) is implemented in terms of primitive operators (`product`,`where`). One notable feature of the implementation is that it uses the previous interpreter `Lproc`. For instance, `Lroc` expressiones are used in `conditional_join` to evaluate to boolean value in `where` clauses. Readers can use the same idea to extend the interpreter such that it returns the value of expression in table. For example, the equivalent construct in sql is `select col_1+col_2 as col_3`.

References:
1. https://www.composingprograms.com/pages/43-declarative-programming.html
2. Database Management Systems 2ed. - Raghu & Gehrke

In [1]:
from Lproc import *
def extend_env_vars(vars,vals,env):
    match [vars,vals]:
        case [(),()]:
            return env
        case [(x,*xs),(v,*vs)]:
            return extend_env(x,v,extend_env_vars(xs,vs,env))

In [2]:
from dataclasses import dataclass

@dataclass
class Table:
    schema:tuple[str]
    rows:tuple[tuple]
    def __post_init__(self):
        self.schema_ref = {s:i for i,s in enumerate(self.schema)}
    
    def __str__(self):
        return (f'{"|".join(self.schema)}\n' 
                + "|".join("-" for _ in self.schema) + "\n"
                + "\n".join("|".join([str(i) for i in r]) for r in self.rows))
    
    def display(self):
        from IPython.display import Markdown,display
        display(Markdown(str(self)))
    
    def project(self,cols:list[str]):
        idx = tuple(self.schema_ref[c] for c in cols)
        return Table(cols,tuple(tuple(r[i] for i in idx) for r in self.rows))
    
    def exclude(self,cols: list[str]):
        return self.project(tuple(c for c in self.schema if c not in set(cols)))
    
    def apply_row(self,exp:Exp,r:tuple):
        env = extend_env_vars(self.schema,r,create_env())
        return interp(expand(exp),env)
    
    def where(self,exp:Exp):
        rows = tuple(r for r in self.rows if self.apply_row(exp,r) is True)
        return Table(self.schema,rows)
    
    def rename(self,mapper:dict[str,str]):
        return Table(tuple(mapper.get(c,c) for c in self.schema),self.rows)
    
    def union(self, t2):
        assert(self.schema == t2.schema)
        return Table(self.schema,self.rows + t2.rows)

    def intersection(self, t2):
        assert(self.schema == t2.schema)
        return Table(self.schema,tuple(set.intersection(set(self.rows),set(t2.rows))))
        
    def difference(self, t2):
        assert(self.schema == t2.schema)
        return Table(self.schema,tuple(set.difference(set(self.rows),set(t2.rows))))

    def product(self, t2):
        "doesn't renaming"
        from itertools import product
        return Table(self.schema + t2.schema, tuple(l+r for l,r in product(self.rows,t2.rows)))

    def conditional_join(self,t2,exp:Exp):
        return self.product(t2).where(exp)
    
    def equijoin(self,t2,keys:list[str]):
        def join_cond(keys:list[str]):
            match keys:
                case []:
                    return True
                case [k,*ks]:
                    return And(Eq(Var(k),Var(f'r.{k}')),join_cond(ks))
        
        renames = {k:f'r.{k}' for k in keys}
        return self.conditional_join(t2.rename(renames),
                                     join_cond(keys)).exclude(list(renames.values()))
    
    def natural_join(self,t2):
        return self.equijoin(t2,list(set.intersection(set(self.schema),t2.schema)))

# Create Table Instances

In [3]:
s1 = Table(('sailor_id','name','rating','age'),
      ((22,'Dustin',7,45),
       (31,'Lubber',8,55),
       (58,'Rusty',10,35),))
s2 = Table(('sailor_id','name','rating','age'),(
    (28,'yuppy',9,35),
    (31,'Lubber',8,55),
    (44,'guppy',5,35),
    (58,'Rusty',10,35),
))
r1 = Table(('sailor_id','boat_id','day'),
           ((22,101,'10/10/96'),
            (58,103,'11/12/96')))
s1.display()
s2.display()
r1.display()

sailor_id|name|rating|age
-|-|-|-
22|Dustin|7|45
31|Lubber|8|55
58|Rusty|10|35

sailor_id|name|rating|age
-|-|-|-
28|yuppy|9|35
31|Lubber|8|55
44|guppy|5|35
58|Rusty|10|35

sailor_id|boat_id|day
-|-|-
22|101|10/10/96
58|103|11/12/96

# Test out the operations

In [4]:
r = s2.project(['name','rating'])
r.display()
r = s2.where(Gt(Var('rating'),7))
r.display()
r = (s2
     .where(Gt(Var('rating'),7))
     .project(['name','rating']))
r.display()
r = s1.union(s2)
r.display()
r = s1.intersection(s2)
r.display()
r = s1.difference(s2)
r.display()
r = s1.product(s2.rename({c:'tmp.' + c for c in s2.schema}))
r.display()

name|rating
-|-
yuppy|9
Lubber|8
guppy|5
Rusty|10

sailor_id|name|rating|age
-|-|-|-
28|yuppy|9|35
31|Lubber|8|55
58|Rusty|10|35

name|rating
-|-
yuppy|9
Lubber|8
Rusty|10

sailor_id|name|rating|age
-|-|-|-
22|Dustin|7|45
31|Lubber|8|55
58|Rusty|10|35
28|yuppy|9|35
31|Lubber|8|55
44|guppy|5|35
58|Rusty|10|35

sailor_id|name|rating|age
-|-|-|-
31|Lubber|8|55
58|Rusty|10|35

sailor_id|name|rating|age
-|-|-|-
22|Dustin|7|45

sailor_id|name|rating|age|tmp.sailor_id|tmp.name|tmp.rating|tmp.age
-|-|-|-|-|-|-|-
22|Dustin|7|45|28|yuppy|9|35
22|Dustin|7|45|31|Lubber|8|55
22|Dustin|7|45|44|guppy|5|35
22|Dustin|7|45|58|Rusty|10|35
31|Lubber|8|55|28|yuppy|9|35
31|Lubber|8|55|31|Lubber|8|55
31|Lubber|8|55|44|guppy|5|35
31|Lubber|8|55|58|Rusty|10|35
58|Rusty|10|35|28|yuppy|9|35
58|Rusty|10|35|31|Lubber|8|55
58|Rusty|10|35|44|guppy|5|35
58|Rusty|10|35|58|Rusty|10|35

In [5]:
r = s1.equijoin(r1,['sailor_id'])
r.display()
r = s1.natural_join(r1)
r.display()

sailor_id|name|rating|age|boat_id|day
-|-|-|-|-|-
22|Dustin|7|45|101|10/10/96
58|Rusty|10|35|103|11/12/96

sailor_id|name|rating|age|boat_id|day
-|-|-|-|-|-
22|Dustin|7|45|101|10/10/96
58|Rusty|10|35|103|11/12/96